In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [3]:
batch_size = 32
num_classes = 10
epochs = 30
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
def VGG(n_ly1, n_ly2):
    model = Sequential()
    model.add(Conv2D(n_ly1, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(n_ly1, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(n_ly2, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(n_ly2, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

#     print("# of params: ", model.count_params())
    return model

# check # of params curve

VGG(2, 4)
VGG(4, 8)
VGG(6, 12)
VGG(8, 16)
VGG(12, 24)
VGG(16, 32)
VGG(20, 40)
VGG(24, 48)
VGG(28, 56)
VGG(32, 64)

# [2, 4, 6, 8, 12, 16, 20, 24, 28, 32]

# of params:  79688
# of params:  154238
# of params:  229292
# of params:  304850
# of params:  457478
# of params:  612122
# of params:  768782
# of params:  927458
# of params:  1088150
# of params:  1250858


In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [14]:
result = {}

result['param_num'] = []
result['train_acc'] = []
result['test_acc'] = []
result['train_loss'] = []
result['test_loss'] = []

In [17]:
layParam = [4, 6, 8, 12, 16, 20, 24, 28, 32]
for itr, params in enumerate(layParam):
    
    model = VGG(params, params*2)
    print('***itr: ', itr, '  |  params: ', model.count_params())
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=30,
                  validation_data=(x_test, y_test),
                  verbose = 0,
    shuffle=True)

    score = model.evaluate(x_train, y_train, verbose=0)
    train_loss, train_acc = score
    score = model.evaluate(x_test, y_test, verbose=0)
    test_loss, test_acc = score


    result['param_num'].append(round(model.count_params(), 4))
    result['train_acc'].append(round(train_acc, 4))
    result['test_acc'].append(round(test_acc, 4))
    result['train_loss'].append(round(train_loss, 4))
    result['test_loss'].append(round(test_loss, 4))

    ### print
    print(
        'train_acc : %.4f' % train_acc,
        '\t test_acc : %.4f' % test_acc,
        '\t train_loss : %.4f' % train_loss,
        '\t test_loss : %.4f' % test_loss,
    )

# of params:  154238
***itr:  0   |  params:  154238
train_acc : 0.5645 	 test_acc : 0.5495 	 train_loss : 1.2565 	 test_loss : 1.2717
# of params:  229292
***itr:  1   |  params:  229292
train_acc : 0.5742 	 test_acc : 0.5639 	 train_loss : 1.2297 	 test_loss : 1.2542
# of params:  304850
***itr:  2   |  params:  304850
train_acc : 0.6427 	 test_acc : 0.6229 	 train_loss : 1.0408 	 test_loss : 1.0859
# of params:  457478
***itr:  3   |  params:  457478
train_acc : 0.7116 	 test_acc : 0.6845 	 train_loss : 0.8435 	 test_loss : 0.9109
# of params:  612122
***itr:  4   |  params:  612122
train_acc : 0.7101 	 test_acc : 0.6741 	 train_loss : 0.8399 	 test_loss : 0.9322
# of params:  768782
***itr:  5   |  params:  768782
train_acc : 0.7590 	 test_acc : 0.7212 	 train_loss : 0.7129 	 test_loss : 0.8124
# of params:  927458
***itr:  6   |  params:  927458
train_acc : 0.7870 	 test_acc : 0.7498 	 train_loss : 0.6291 	 test_loss : 0.7425
# of params:  1088150
***itr:  7   |  params:  1088150


In [18]:
result

{'param_num': [79688,
  154238,
  229292,
  304850,
  457478,
  612122,
  768782,
  927458,
  1088150,
  1250858],
 'test_acc': [0.43590000000000001,
  0.54949999999999999,
  0.56389999999999996,
  0.62290000000000001,
  0.6845,
  0.67410000000000003,
  0.72119999999999995,
  0.74980000000000002,
  0.73909999999999998,
  0.76790000000000003],
 'test_loss': [1.5630999999999999,
  1.2717000000000001,
  1.2542,
  1.0859000000000001,
  0.91090000000000004,
  0.93220000000000003,
  0.81240000000000001,
  0.74250000000000005,
  0.78100000000000003,
  0.68540000000000001],
 'train_acc': [0.43319999999999997,
  0.5645,
  0.57420000000000004,
  0.64270000000000005,
  0.71160000000000001,
  0.71009999999999995,
  0.75900000000000001,
  0.78700000000000003,
  0.7843,
  0.82520000000000004],
 'train_loss': [1.5609,
  1.2565,
  1.2297,
  1.0407999999999999,
  0.84350000000000003,
  0.83989999999999998,
  0.71289999999999998,
  0.62909999999999999,
  0.64319999999999999,
  0.55759999999999998]}

In [20]:
import pickle

with open("./output/result_CIFAR10.pkl", "wb") as file:
    pickle.dump(result, file)